In [1]:
!module list

Currently Loaded Modulefiles:
  1) /switcher/1.0.15              2) gcc-9.3.0-gcc-7.3.0-fjzqkyt


In [ ]:
# Set up latest torch instance with Python=3.9
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from datasets import load_dataset
from pathlib import Path
import json
from datetime import datetime

In [3]:
torch.cuda.is_available()

True

### Select Model

In [4]:
root_dir = Path("..")
model_id = root_dir / "weights" / "weights" / "13Bf_hf" # location for model directory, must be in torch format
tokenizer = LlamaTokenizer.from_pretrained(model_id)

In [5]:
model = LlamaForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.float16) # device_map='auto' will use GPU if available

Loading checkpoint shards: 100%|██████████| 6/6 [00:51<00:00,  8.55s/it]


### Sys Prompt and Training Data

In [6]:
sys_prompt = """Pretend you are an observant Assistant, a helpful bot that takes course catalogue data and returns clean JSON object list. If available, the JSON objects should contain for each course in the text a "Number", "Title", "Description", "Prerequisite", and "Credits". There can be multiple Course Numbers and multiple Prerequisites, include all of them.  If the text is not a course description, return "[]"."""

In [7]:
train_dataset = load_dataset('json', data_files = str(root_dir / 'data' / 'raw' / 'training.json'), split='train')  

# split train into train and validation
train_dataset = train_dataset.train_test_split(test_size=0.09, shuffle=True)

In [8]:
# The correct json format-
train_dataset['train'][0]

{'output': '[\n    {\n        "Number": [\n            171\n        ],\n        "Title": "Intro to Psycholinguistics",\n        "Description": "Psycholinguistics studies the cognitive processes involved in acquiring, understanding, and producing language. Speech perception, word recognition, and sentence processing are some of the topics covered",\n        "Prerequisites": "LING 080 or PSYS 001",\n        "Credits": "3"\n    },\n    {\n        "Number": [\n            174\n        ],\n        "Title": "Language & Gender",\n        "Description": "Examines the many ways that gendered identities are socially and linguistically constructed through a variety of sociocultural practices, with specific attention throughout on interrogating how language relates to systems of power and oppression",\n        "Prerequisites": "LING 035 or LING 080 or GSWS 001",\n        "Credits": "3"\n    },\n    {\n        "Number": [\n            175\n        ],\n        "Title": "Language, Gender and Sexualit

In [9]:
def formatting_func(example):
    text = f"""<s>[INST] <<SYS>> {sys_prompt} <</SYS>>

```{example['input']}``` 
Make sure to include prerequisites and exclude any 
non-course information. [/INST] {example['output']}
"""
    return text

In [10]:
# formatted data
print(formatting_func(train_dataset['train'][0]))

<s>[INST] <<SYS>> Pretend you are an observant Assistant, a helpful bot that takes course catalogue data and returns clean JSON object list. If available, the JSON objects should contain for each course in the text a "Number", "Title", "Description", "Prerequisite", and "Credits". There can be multiple Course Numbers and multiple Prerequisites, include all of them.  If the text is not a course description, return "[]". <</SYS>>

```duction to Pragmatics. 3 Credits.
An exploration of the contexts of language--physical, linguistic, and
cultural--and their roles in determining the meaning of everyday talk
and writing. Pre/co-requisites: LING 080.LING 170. TESOL and Applied Linguistics. 3 Credits.
Provides an overview of second language/ESL classroom theory and
research. Topics include: teaching approaches, learning environment
and outcomes, program planning, syllabus and material design, lesson
planning, and assessment. Emphasis on practical application of topics
discussed. Prerequisite: 

In [11]:
def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt)) # tokenize the data

In [12]:
tokenized_train_dataset = train_dataset["train"].map(generate_and_tokenize_prompt)
tokenized_validate_dataset = train_dataset["test"].map(generate_and_tokenize_prompt)

Map: 100%|██████████| 25/25 [00:00<00:00, 143.01 examples/s]


In [14]:
##### Uncomment if you want to evaluate first ######

# eval_prompt = f"""<<SYS>>{sys_prompt}<</SYS>>

# [INST]Text: {tokenized_validate_dataset[10]['input']}[/INST]
# """

# model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# model.eval()
# with torch.no_grad():
#     print(tokenizer.decode(model.generate(**model_input, max_new_tokens=1000)[0], skip_special_tokens=True))

In [13]:
tokenizer.pad_token = tokenizer.eos_token # llama quirk, have to do it
model.gradient_checkpointing_enable() # makes the training faster

### Set up Model Training with LoRa Config

In [14]:
model.train() # put the model in training mode

def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
    )
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=256, # tune this
        lora_alpha=512, # and this
        lora_dropout=0.05,
        # the target modules can also be tuned
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
            "lm_head",
        ]
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config
try:
    # create peft config
    model, lora_config = create_peft_config(model)
except:
    model, lora_config = create_peft_config(model)

trainable params: 1,010,892,800 || all params: 14,026,757,120 || trainable%: 7.20688888637433


In [15]:
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
output_dir = "tmp/llama-output"

# also tune-able
config = {
    'lora_config': lora_config,
    'learning_rate': 2.5e-5,
    'num_train_epochs': 2, # especially this one
    'gradient_accumulation_steps': 1,
    'per_device_train_batch_size': 1,
    'gradient_checkpointing': True,
}

# Set up profiler, and connect to wandb.ai
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

### Train the Model, monitor on wandb

In [ ]:
from transformers import default_data_collator, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    fp16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    evaluation_strategy="steps",
    eval_steps=10,
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset= tokenized_validate_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
        callbacks=[profiler_callback] if enable_profiler else [],
    )
    # Start training
    trainer.train()

### Wrap-up

In [ ]:
# merge the weights, otherwise it will only save as lora weights, which llama.cpp does not support atm
model = model.merge_and_unload()

In [20]:
# save the model
model.save_pretrained("../outputs/13Bf_finetuned_01")
tokenizer.save_pretrained("../outputs/13Bf_finetuned_01")

In [22]:
?LlamaForCausalLM.from_pretrained

Signature:
LlamaForCausalLM.from_pretrained(
    pretrained_model_name_or_path: Union[str, os.PathLike, NoneType],
    *model_args,
    config: Union[transformers.configuration_utils.PretrainedConfig, str, os.PathLike, NoneType] = None,
    cache_dir: Union[str, os.PathLike, NoneType] = None,
    ignore_mismatched_sizes: bool = False,
    force_download: bool = False,
    local_files_only: bool = False,
    token: Union[str, bool, NoneType] = None,
    revision: str = 'main',
    use_safetensors: bool = None,
    **kwargs,
)
Docstring:
Instantiate a pretrained pytorch model from a pre-trained model configuration.

The model is set in evaluation mode by default using `model.eval()` (Dropout modules are deactivated). To train
the model, you should first set it back in training mode with `model.train()`.

The warning *Weights from XXX not initialized from pretrained model* means that the weights of XXX do not come
pretrained with the rest of the model. It is up to you to train those weigh

In [ ]:
model = LlamaForCausalLM.from_pretrained(root_dir / "outputs" / "13Bf_finetuned_01", device_map='auto', torch_dtype=torch.float16)

In [21]:
# Evaluate the new model
eval_prompt = f"""<<SYS>>{sys_prompt}<</SYS>>

[INST]Text: {tokenized_validate_dataset[10]['input']}[/INST]
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=1000)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<<SYS>>You are a helpful Assistant, you take course catalogue raw text extract and return a clean JSON object list. If available, the JSON objects should contain for each course in the text a "Number", "Title", "Description", "Prerequisites", and "Credits". There can be multiple Course Numbers include all of them. If the text is not a course description, return '[]'.<</SYS>>

[INST]Text: oselective and
enantioselective processes. Prerequisite: CHEM 241.
CHEM 260. Advanced Physical Chemistry. 3 Credits.
Builds on the concepts from Introductory Physical Chemistry
(CHEM 165). The three major areas of quantum chemistry,
thermodynamics, and kinetics are extended in greater depth, and at
a higher level of mathematical rigor. Prerequisite: CHEM 165. Co-
requisites: CHEM 167 or MATH 121.
CHEM 267. Topics in Physical Chemistry. 1-3 Credits.
Selected topics of current interest in physical chemistry. See Schedule
of Courses for specific titles. May be repeated for credit with different
content. P